In [16]:
import numpy as np
import pandas as pd
#import seaborn as sns
from glob import glob
from PIL import Image
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
from imutils import paths
import random

from subprocess import check_output

import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.applications.densenet import preprocess_input
from keras.preprocessing.image import (random_rotation, random_shift, random_shear, random_zoom)


In [30]:
# load a single image to np array
def get_image(img_path):
    img = load_img(img_path, target_size=(224, 224))#.convert('L')
    img = img_to_array(img)
    img = img/255
    return img

def to_one_hot(labels, preClass2id = None, preId2class = None):
    if preId2class is None:
        id2class = dict()
        class2id = dict()      ##!!! class(integer) to id(whale class)
        num_labels = []
        counter = 0
    else:
        id2class = preId2class
        class2id = preClass2id
        num_labels = []
        counter = len(class2id.keys())
    for id in labels:
        if id not in id2class.keys():
            num_labels.append(counter)
            id2class[id] = counter
            class2id[counter] = id
            counter += 1
        else:
            num_labels.append(id2class[id])
    one_hot_labels = to_categorical(num_labels, num_classes = 4251)
    return one_hot_labels, class2id, id2class, num_labels


train_label_dir = '/home/jiaruizou/Desktop/CS543Final/train.csv'
df = pd.read_csv(train_label_dir)

labels = df['Id']
labels, class2id, id2class, num_labels = to_one_hot(labels)

# generate class weights
from sklearn.utils import class_weight
cw = class_weight.compute_class_weight('balanced', np.arange(len(class2id)), np.asarray(num_labels))
cw = dict(enumerate(cw.flatten(), 1))
# since cw start at 1 instead of 0, we need to offset this by 1
cw_z = dict()
for key in cw:
    cw_z[key-1] = cw[key]

In [18]:
# loading train images
trainPaths = list(paths.list_images('/home/jiaruizou/Desktop/CS543Final/train/'))
print(len(trainPaths))
train = np.array([get_image(img_path) for img_path in trainPaths])
print(train.shape)

9081
(9081, 224, 224, 3)


In [41]:
# split the data into train and val
# The dataset is super unbalanced, as there are many classes that contains only 1 image
# As a result, train/val data cannot be split before generating more data by augmentation
train_dir = '/home/jiaruizou/Desktop/CS543Final/train/'

num_categories = len(df['Id'].unique())
#print(num_categories)
validation = np.zeros((num_categories, 224, 224, 3))
validation_y = []

i = 0
for id in df['Id'].unique():
    validation_y.append(id)
    im = df[df['Id'] == id].sample(1)
    name =  np.array(im.get('Image'))[0]
    im = get_image(train_dir + name)

    # https://www.kaggle.com/lextoumbourou/humpback-whale-id-data-and-aug-exploration
    x = -1#random.randint(0, 3)
    if x == -1:
        validation[i,:,:,:] = im # for debugging
    if x == 0:
        validation[i,:, :, :] = random_rotation(im, 30, row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest')
    if x == 1:
        validation[i,:, :, :] = random_shift(im, wrg=0.1, hrg=0.3, row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest')
    if x == 2:
        validation[i,:, :, :] = random_shear(im, intensity=0.4, row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest')
    if x == 3:
        validation[i,:, :, :] = random_zoom(im, zoom_range=(1.5, 0.7), row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest')
    i = i + 1

validation_y = to_one_hot(validation_y, class2id, id2class)[0]

In [20]:
# define data generator
#use of an image generator for preprocessing and data augmentation
image_gen = ImageDataGenerator(
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    #rescale=1./255,
    rotation_range=15,
    width_shift_range=.15,
    height_shift_range=.15,
    horizontal_flip=True
)

In [47]:
val_gen = ImageDataGenerator()
train_gen = ImageDataGenerator()

In [27]:
#train.shape

(9081, 224, 224, 3)

In [46]:
# acquire the model
model = keras.applications.densenet.DenseNet121(include_top=True, weights=None, classes=4251)

In [ ]:
batch_size = 32
epochs = 200

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_dir = '/home/jiaruizou/Desktop/CS543Final/saved_model/DenseNet/'
routine_dir = model_dir + "routine-{epoch:02d}-{acc:.2f}.hdf5"
routine_save = ModelCheckpoint(routine_dir, monitor='acc', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=10)
best_dir = model_dir + "best-{epoch:02d}-{acc:.2f}.hdf5"
best_save = ModelCheckpoint(best_dir, monitor='acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=4)


hist = model.fit_generator(train_gen.flow(train, labels, batch_size=batch_size),
                    steps_per_epoch=train.shape[0]//batch_size,
                    epochs=epochs, verbose=1, callbacks=[routine_save, best_save], class_weight=cw_z,
                    #       validation_data=(validation, validation_y))
                    validation_data=val_gen.flow(validation, validation_y, batch_size=batch_size),
                           validation_steps=validation.shape[0]//batch_size)


Epoch 1/200
283/283 [==============================] - 136s 480ms/step - loss: 8.4782 - acc: 1.1042e-04 - val_loss: 8.6319 - val_acc: 4.7348e-04
Epoch 2/200
283/283 [==============================] - 94s 331ms/step - loss: 8.2969 - acc: 0.0000e+00 - val_loss: 9.3698 - val_acc: 2.3674e-04
Epoch 3/200
283/283 [==============================] - 95s 334ms/step - loss: 7.9898 - acc: 6.6255e-04 - val_loss: 9.2083 - val_acc: 0.0000e+00
Epoch 4/200
283/283 [==============================] - 94s 333ms/step - loss: 7.5328 - acc: 7.7297e-04 - val_loss: 9.6418 - val_acc: 0.0000e+00
Epoch 5/200
283/283 [==============================] - 94s 331ms/step - loss: 7.0542 - acc: 0.0021 - val_loss: 10.1663 - val_acc: 0.0000e+00
Epoch 6/200
283/283 [==============================] - 94s 333ms/step - loss: 6.5584 - acc: 0.0044 - val_loss: 10.8190 - val_acc: 0.0000e+00
Epoch 7/200
283/283 [==============================] - 95s 334ms/step - loss: 6.0551 - acc: 0.0119 - val_loss: 11.5490 - val_acc: 0.0000e+00


In [ ]:
# loading test images
testPaths = list(paths.list_images('/home/jiaruizou/Desktop/CS543Final/test/'))
print(len(testPaths))
test_images = np.array([get_image(img_path) for img_path in testPaths])
print(test_images.shape)

In [ ]:
# loading trained model
model_name = 'routine-170-0.99'
model = load_model('/home/jiaruizou/Desktop/CS543Final/saved_model/DenseNet/' + model_name+ '.hdf5')

In [ ]:
# predict!
pred = model.predict(test_images)

# testing and generating submission file
import warnings
from os.path import split

pred_dir = "/home/jiaruizou/Desktop/CS543Final/prediction/DenseNet/"

with open(pred_dir + model_name + ".csv", "w") as f:
    f.write("Image,Id\n")
    top_5 = np.argsort(pred)[:,-1:-6:-1]   # get the top 5 most likely classes
    for i in range(top_5.shape[0]):
        cur_tags = ''
        cur_image_name = testPaths[i].split('/')[-1]
        for j in range(5):
            cur_tags = cur_tags + ' ' + class2id[top_5[i][j]]
        f.write("%s,%s\n" %(cur_image_name, cur_tags))     

In [ ]:
from util import plotHistory
plotHistory.plotTrainingHistory(hist)